In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_vgg19
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

# CIFAR-10類別名稱
CIFAR10_CLASSES = [
    '飛機', '汽車', '鳥', '貓', '鹿',
    '狗', '青蛙', '馬', '船', '卡車'
]

# 數據預處理函數
def preprocess_cifar10_data(num_samples=2500):
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    # 隨機選取 num_samples 張圖像
    X_train, y_train = X_train[:num_samples], y_train[:num_samples]
    X_test, y_test = X_test[:num_samples], y_test[:num_samples]

    # 圖像大小調整函數
    def resize_images(images):
        return tf.image.resize(images, (224, 224))

    X_train_resized = resize_images(X_train)
    X_test_resized = resize_images(X_test)

    return X_train_resized, y_train, X_test_resized, y_test


# 創建VGG模型微調版本
def create_vgg_model(base_model, num_classes=10):
    # 凍結基礎模型層
    for layer in base_model.layers:
        layer.trainable = False

    # 添加自定義分類層
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # 創建最終模型
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# 訓練模型
def train_vgg_model(model, X_train, y_train, X_test, y_test, preprocess_func):
    # 動態調整圖像大小並進行預處理
    def preprocess_and_resize(images, preprocess_func):
        resized_images = tf.image.resize(images, (224, 224))
        return preprocess_func(resized_images)

    X_train_processed = preprocess_and_resize(X_train, preprocess_func)
    X_test_processed = preprocess_and_resize(X_test, preprocess_func)

    y_train_encoded = to_categorical(y_train, 10)
    y_test_encoded = to_categorical(y_test, 10)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        X_train_processed, y_train_encoded,
        epochs=10, batch_size=32,
        validation_data=(X_test_processed, y_test_encoded),
        verbose=1
    )

    return history, X_test_processed, y_test_encoded


# 可視化預測結果
def visualize_predictions(model, X_test, y_test, model_name):
    # 進行預測
    predictions = model.predict(X_test)

    # 設置繪圖
    plt.figure(figsize=(15, 3))

    # 隨機選擇10個樣本
    for i in range(10):
        # 隨機選擇一個測試樣本
        idx = np.random.randint(0, len(X_test))

        # 準備子圖
        plt.subplot(2, 5, i+1)

        # 顯示原始圖像（需要反標準化）
        img = X_test[idx].astype('float32')
        plt.imshow(img.astype('uint8'))

        # 獲取預測和真實標籤
        pred_label = np.argmax(predictions[idx])
        true_label = y_test[idx][0]

        # 設置標題
        color = 'green' if pred_label == true_label else 'red'
        plt.title(f'{model_name}\nPredic: {CIFAR10_CLASSES[pred_label]}\nTrue: {CIFAR10_CLASSES[true_label]}',
                  color=color, fontsize=8)
        plt.axis('off')

    plt.suptitle(f'{model_name} Model ', fontsize=16)
    plt.tight_layout()
    plt.show()

# 主函數
def main():
    # 預處理數據
    X_train, y_train, X_test, y_test = preprocess_cifar10_data()

    # 創建VGG16模型
    print("VGG16 模型:")
    vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg16_model = create_vgg_model(vgg16_base)
    vgg16_model.summary()

    # 訓練VGG16模型
    vgg16_history, X_test_vgg16, y_test_vgg16 = train_vgg_model(
        vgg16_model, X_train, y_train, X_test, y_test, preprocess_vgg16
    )

    # 創建VGG19模型
    print("\nVGG19 模型:")
    vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg19_model = create_vgg_model(vgg19_base)
    vgg19_model.summary()

    # 訓練VGG19模型
    vgg19_history, X_test_vgg19, y_test_vgg19 = train_vgg_model(
        vgg19_model, X_train, y_train, X_test, y_test, preprocess_vgg19
    )

    # 評估模型
    print("\n模型性能評估:")
    print("VGG16 模型:")
    vgg16_accuracy = vgg16_model.evaluate(preprocess_vgg16(X_test), to_categorical(y_test, 10))[1]
    print(f'準確率: {vgg16_accuracy * 100:.2f}%')

    print("\nVGG19 模型:")
    vgg19_accuracy = vgg19_model.evaluate(preprocess_vgg19(X_test), to_categorical(y_test, 10))[1]
    print(f'準確率: {vgg19_accuracy * 100:.2f}%')

    # 可視化預測結果
    print("\n正在可視化預測結果...")
    visualize_predictions(vgg16_model, X_test_vgg16, y_test_vgg16, 'VGG16')
    visualize_predictions(vgg19_model, X_test_vgg19, y_test_vgg19, 'VGG19')

    return vgg16_model, vgg19_model

# 執行主函數
if __name__ == '__main__':
    vgg16_model, vgg19_model = main()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
VGG16 模型:
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         525,3

 Total params: 15,250,250 (58.18 MB)

 Trainable params: 535,562 (2.04 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 20,559,946 (78.43 MB)

 Trainable params: 535,562 (2.04 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 31s 341ms/step - accuracy: 0.4345 - loss: 2.4566 - val_accuracy: 0.7772 - val_loss: 0.6840
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 36s 301ms/step - accuracy: 0.7841 - loss: 0.6463 - val_accuracy: 0.7880 - val_loss: 0.6336
Epoch 3/10
55/79 ━━━━━━━━━━━━━━━━━━━━ 3s 152ms/step - accuracy: 0.8339 - loss: 0.4650